# Tavily搜索结果（已弃用）

:::info 弃用通知

此工具已被弃用。请改用 `@langchain/tavily` 包中的 [TavilySearch](./tavily_search) 工具。

:::

[Tavily](https://tavily.com/) 搜索是一个专为LLM代理定制的强大搜索API。它能够与各种数据源无缝集成，以确保更高级别、更相关的搜索体验。

本指南提供了开始使用 Tavily [工具](/docs/integrations/tools/) 的快速概览。如需关于 Tavily 工具的完整详细文档，您可以在 [API 参考文档](https://v03.api.js.langchain.com/classes/_langchain_community_tools_tavily_search.TavilySearchResults.html) 中找到。

## 概览

### 集成详情

| 类名 | 包名 | [Python支持](https://python.langchain.com/docs/integrations/tools/tavily_search/) | 最新包版本 |
| :--- | :--- | :---: | :---: |
| [TavilySearchResults](https://api.js.langchain.com/classes/langchain_community_tools_tavily_search.TavilySearchResults.html) | [`@langchain/community`](https://www.npmjs.com/package/@langchain/community) | ✅ |  ![NPM - 版本](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

## 安装配置

该集成在 `@langchain/community` 包中，您可以按如下方式安装：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/core
</Npm2Yarn>
```

### 凭证

在此处 [链接](https://app.tavily.com) 设置一个API密钥，并将其设置为名为 `TAVILY_API_KEY` 的环境变量。

```typescript
process.env.TAVILY_API_KEY = "YOUR_API_KEY"
```

另外，设置 [LangSmith](https://smith.langchain.com/) 以获得最佳观测能力也是有帮助的（但不是必需的）：

```typescript
process.env.LANGSMITH_TRACING="true"
process.env.LANGSMITH_API_KEY="your-api-key"
```

## 实例化

你可以通过以下方式导入并实例化 `TavilySearchResults` 工具：

In [1]:
import { TavilySearchResults } from "@langchain/community/tools/tavily_search";

const tool = new TavilySearchResults({
  maxResults: 2,
  // ...
});

## 调用

### [使用参数直接调用](/docs/concepts/tools)

你可以像这样直接调用工具：

In [ ]:
await tool.invoke({
  input: "what is the current weather in SF?",
});

### [使用工具调用调用](/docs/concepts/tools)

我们还可以使用模型生成的 `ToolCall` 来调用工具，在这种情况下将返回 `ToolMessage`：

In [ ]:
// This is usually generated by a model, but we'll create a tool call directly for demo purposes.
const modelGeneratedToolCall = {
  args: {
    query: "what is the current weather in SF?"
  },
  id: "1",
  name: tool.name,
  type: "tool_call",
}

await tool.invoke(modelGeneratedToolCall)

## 链式调用

我们可以通过首先将其绑定到一个[工具调用模型](/docs/how_to/tool_calling/)，然后调用它，以实现链式使用：

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```


In [4]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai"

const llm = new ChatOpenAI({
  model: "gpt-4o",
  temperature: 0,
})

In [7]:
import { HumanMessage } from "@langchain/core/messages";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnableLambda } from "@langchain/core/runnables";

const prompt = ChatPromptTemplate.fromMessages(
  [
    ["system", "You are a helpful assistant."],
    ["placeholder", "{messages}"],
  ]
)

const llmWithTools = llm.bindTools([tool]);

const chain = prompt.pipe(llmWithTools);

const toolChain = RunnableLambda.from(
  async (userInput: string, config) => {
    const humanMessage = new HumanMessage(userInput,);
    const aiMsg = await chain.invoke({
      messages: [new HumanMessage(userInput)],
    }, config);
    const toolMsgs = await tool.batch(aiMsg.tool_calls, config);
    return chain.invoke({
      messages: [humanMessage, aiMsg, ...toolMsgs],
    }, config);
  }
);

const toolChainResult = await toolChain.invoke("what is the current weather in sf?");

In [ ]:
const { tool_calls, content } = toolChainResult;

console.log("AIMessage", JSON.stringify({
  tool_calls,
  content,
}, null, 2));

## 代理

关于如何在代理中使用LangChain工具的指南，请参阅 [LangGraph.js](https://langchain-ai.github.io/langgraphjs/) 文档。

## API 参考文档

如需详细了解所有 `TavilySearchResults` 的功能和配置，请前往 API 参考文档：

https://api.js.langchain.com/classes/langchain_community_tools_tavily_search.TavilySearchResults.html

## 相关内容

* 工具[概念指南](https://js.langchain.com/docs/concepts/tools/)

* 工具[操作指南](https://js.langchain.com/docs/how_to/#tools)